# Overview Materi

Jelaskan perbedaan singkat antara grid, randomized, bayesian search cv dengan optuna menurut pemahamanmu

source: https://www.youtube.com/watch?v=t-INgABWULw

Grid : pendekatan penyelesaian hyperparameter dengan menelusuri segala kombinasi nilai hiperparameter untuk menemukan kumpulan parameter terbaik

Randomized : pendekatan penyelesaian hyperparameter dengan mengambil sampel hiperparameter secara acak untuk menemukan parameter terbaik

Bayesian search CV : pendekatan penyelesaian hyperparameter dengan enambah atau mengurangi fungsi objektif dengan komputasi minimal

Optuna : pendekatan penyelesaian hyperparameter dengan mengeksplorasi ruang hiperparameter berdasarkan evaluasi sebelumnya secara cerdas

# Import Data & Libraries

In [1]:
# jalankan hanya sekali
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 4.4 MB/s eta 0:00:00


In [2]:
# import library yang dibutuhkan di sini
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import optuna

In [3]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# Data Preprocessing

In [4]:
# ubah variabel kategorik ke numerik
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['species'] = le.fit_transform(df['species'])
display(df.head())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
# subsetting peubah
X = df.drop(['species'], axis=1)
y = df['species']

# Dataset Splitting

In [6]:
# split dengan rasio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Base Model Random Forest

In [7]:
# gunakan random forest classifier
rfr = RandomForestClassifier(random_state=42)
rfr.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
y_pred = rfr.predict(X_test)

In [9]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 1.000
Precision: 1.000
Recall: 1.000
F1 Score: 1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Optuna

In [25]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

    return score.mean()

Hyperparameter dapat disesuaikan dengan algoritma yang digunakan. Kali ini kita menggunakan Random Forest sehingga yang dapat kita select adalah *n_estimators, max_depth, min_samples_split,* dan *min_samples_leaf*

In [22]:
study = optuna.create_study(direction='maximize')

[I 2025-10-04 12:37:54,156] A new study created in memory with name: no-name-54bc9f2c-7866-4b0e-8442-03e01c708b8a


In [28]:
study.optimize(objective, n_trials=100)

[W 2025-10-04 12:41:21,307] Trial 4 failed with parameters: {'n_estimators': 400, 'max_depth': 8, 'min_samples_split': 17, 'min_samples_leaf': 7} because of the following error: NameError("name 'cross_val_score' is not defined").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-145632157.py", line 15, in objective
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
            ^^^^^^^^^^^^^^^
NameError: name 'cross_val_score' is not defined
[W 2025-10-04 12:41:21,316] Trial 4 failed with value None.


NameError: name 'cross_val_score' is not defined

it may take a while... so just wait n see ^^
<br>
they recommend to set n_trials at 100 cz it seems there's no significant score increase after 100 trials (also inefficient too, you'll have to wait in a quite long time)

ValueError: No trials are completed yet.

Berikut hasil hyperparameter tuning dari Optuna

In [31]:
# cek hasil hyperparameter tuning dari Optuna
study.best_params

ValueError: No trials are completed yet.

# Random Forest Using Optuna

In [ ]:
# simpan hasil best hyperparameter tuning ke variabel bari
...

In [ ]:
best_model = ...

best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=17, min_samples_leaf=15, min_samples_split=12,
                       n_estimators=523)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.933
Precision: 0.947
Recall: 0.933
F1 Score: 0.934
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.80      1.00      0.89         8
           2       1.00      0.83      0.91        12

    accuracy                           0.93        30
   macro avg       0.93      0.94      0.93        30
weighted avg       0.95      0.93      0.93        30



Tidak terdapat kenaikan skor dengan sebelum menggunakan Optuna sebab skor yang dihasilkan melalui base model saja sudah bagus